In [35]:
#IMPORTS
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import math
import matplotlib.cm as cm

from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, KFold, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import make_scorer,r2_score
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error


def Visualization(results, save = False, filename = "", min = 0, max = 50,name = "", names = []):
    if names == []:
      names = ['clip_vitb16', 'clip_vitb32', 'clip_vitl14', 'dino_vits', 'dino_vitb', 'dino_vitl', 'dino_vitg', 'sam', 'nasa', 'twopoint']
    medians = np.median(results,axis = 1)

    plt.figure(figsize=(10, 5))

    ax = sns.boxplot(data = results)

    for xtick in ax.get_xticks():
        ax.text(xtick, medians[xtick], f'{medians[xtick]:.2f}',
                    horizontalalignment='center', size='small', color='black')


    ax.set_xticklabels(names)
    ax.set_xlabel('Model')
    ax.set_ylabel('MAPE')
    ax.set_title(name)
    ax.set_ylim(min,max)

    plt.tight_layout()
    if save == True:
        plt.savefig(filename, dpi=300)
    plt.show()

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

def Visualize_Mean(results, save=False, filename="", min=0, max=50, Name="", names=[]):
    if names == []:
        names = ['clip_vitb16', 'clip_vitb32', 'clip_vitl14',
                 'dino_vits', 'dino_vitb', 'dino_vitl',
                 'dino_vitg', 'sam', 'nasa', 'twopoint']

    # Calculate mean and standard deviation
    means = np.mean(results, axis=1)
    std_devs = np.std(results, axis=1)

    # Generate unique colors for markers
    colors = cm.tab10(np.linspace(0, 1, len(names)))

    plt.figure(figsize=(5,10))

    # Plot error bars and colored circles
    for i, (name, mean, std_dev) in enumerate(zip(names, means, std_devs)):
        plt.errorbar(x=mean, y=i, xerr=std_dev, fmt='o', capsize=10,
                     ecolor='black', color = colors[i], elinewidth=1, markersize=10)

        plt.text(mean, i + 0.5, f'{mean:.2f}', verticalalignment='top',
                 horizontalalignment='center', size='medium', color='black')


    # Add labels, title, and customize x-axis ticks
    plt.yticks(ticks=range(len(names)), labels=names)  # Set names and rotate labels
    plt.xlabel('MAPE')
    plt.ylabel('Model')
    plt.title(Name)
    plt.xlim(min, max)

    # Adjust layout and save/show the plot
    plt.tight_layout()
    if save:
        plt.savefig(filename, dpi=300)
    plt.show()

In [3]:
#Just Images
clip_vitb16_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_pr_4.npy')
clip_vitb32_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_pr_5.npy')
clip_vitl14_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_pr_5.npy')
dino_vits_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_pr_4.npy')
dino_vitb_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_pr_3.npy')
dino_vitl_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_pr_2.npy')
dino_vitg_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_pr_4.npy')
sam_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_pr_4.npy')
nasa_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_pr_2.npy')
tp_final_mape = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_pr_1.npy')

#Images + Compositions
clip_vitb16_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_pr_comp_5.npy')
clip_vitb32_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_pr_comp_5.npy')
clip_vitl14_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_pr_comp_5.npy')
dino_vits_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_pr_comp_3.npy')
dino_vitb_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_pr_comp_2.npy')
dino_vitl_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_pr_comp_2.npy')
dino_vitg_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_pr_comp_4.npy')
sam_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_pr_comp_4.npy')
nasa_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_pr_comp_2.npy')
tp_final_mape_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_pr_comp_1.npy')

In [ ]:
print ("Just Images")
print("CLIP VitB16",mean(clip_vitb16_final_mape))
print("CLIP VitB32",mean(clip_vitb32_final_mape))
print("CLIP VitL14",mean(clip_vitl14_final_mape))
print("DINO Vits",mean(dino_vits_final_mape))
print("DINO VitB",mean(dino_vitb_final_mape))
print("DINO VitL",mean(dino_vitl_final_mape))
print("DINO VitG",mean(dino_vitg_final_mape))
print("SAM",mean(sam_final_mape))
print("NASA",mean(nasa_final_mape))
print("Two Point",mean(tp_final_mape))

print ("Images & Compositions")
print("CLIP VitB16",mean(clip_vitb16_final_mape_comp))
print("CLIP VitB32",mean(clip_vitb32_final_mape_comp))
print("CLIP VitL14",mean(clip_vitl14_final_mape_comp))
print("DINO Vits",mean(dino_vits_final_mape_comp))
print("DINO VitB",mean(dino_vitb_final_mape_comp))
print("DINO VitL",mean(dino_vitl_final_mape_comp))
print("DINO VitG",mean(dino_vitg_final_mape_comp))
print("SAM",mean(sam_final_mape_comp))
print("NASA",mean(nasa_final_mape_comp))
print("Two Point",mean(tp_final_mape_comp))

In [ ]:
#IMAGES ONLY


names = ['clip_vitb16_PR',  'clip_vitl14_LR','clip_vitb16_SVR','dino_vitb_PR', 'dino_vitl_LR', 'dino_vitl_SVR', 'sam_PR', 'sam_LR','sam_SVR','nasa_PR', 'nasa_LR', 'nasa_SVR','twopoint_PR', 'twopoint_LR', 'twopoint_SVR']


best_image_results = [clip_vitb16_final_mape,clip_vitl14_lr,clip_vitb16_svr, \
                      dino_vitb_final_mape, dino_vitl_lr,dino_vitl_svr, \
                      sam_final_mape,sam_lr,sam_svr,\
                      nasa_final_mape,nasa_lr,nasa_svr,\
                      tp_final_mape,tp_lr,tp_svr]

Visualize_Mean(best_image_results, names = names,save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/IMAGES.pdf",Name = "Just Images")

In [ ]:
print (sam_svr_comp)
print (np.mean(sam_svr_comp))
print (np.std(sam_svr_comp))
new_sam = np.concatenate((sam_svr_comp[:2],sam_svr_comp[3:]))
print (new_sam)
print (np.mean(new_sam))
print (np.std(new_sam))

In [ ]:
names = ['clip_vitb16_PR', 'clip_vitl14_LR', 'clip_vitb16_SVR', 'dino_vitb_PR', 'dino_vitl_LR', 'dino_vitl_SVR', 'sam_PR', 'sam_LR','sam_SVR','nasa_PR', 'nasa_LR','nasa_SVR',\
         'twopoint_PR', 'twopoint_LR' ,'twopoint_SVR']

best_comp_results = [clip_vitb16_final_mape_comp,clip_vitl14_lr_comp,clip_vitb16_svr_comp,dino_vitb_final_mape_comp, dino_vitl_lr_comp,dino_vitl_svr_comp,\
                     sam_final_mape_comp,sam_lr_comp,sam_svr_comp, nasa_final_mape_comp,nasa_lr_comp, nasa_svr_comp,tp_final_mape_comp, tp_lr_comp, tp_svr_comp]

Visualize_Mean(best_comp_results, names = names,save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/IMAGES_COMPS.pdf",Name = "BOTH")

In [ ]:
pr_results = [clip_vitb16_final_mape,clip_vitb32_final_mape,clip_vitl14_final_mape,dino_vits_final_mape,dino_vitb_final_mape, dino_vitl_final_mape, dino_vitg_final_mape, sam_final_mape,nasa_final_mape,tp_final_mape]
pr_results_comps = [clip_vitb16_final_mape_comp,clip_vitb32_final_mape_comp,clip_vitl14_final_mape_comp,dino_vits_final_mape_comp,dino_vitb_final_mape_comp, dino_vitl_final_mape_comp, dino_vitg_final_mape_comp, sam_final_mape_comp,nasa_final_mape_comp,tp_final_mape_comp]

best_pr_results = [clip_vitb16_final_mape,dino_vitb_final_mape, sam_final_mape,nasa_final_mape,tp_final_mape]
best_pr_results_comps = [clip_vitb16_final_mape_comp,dino_vitb_final_mape_comp, sam_final_mape_comp,nasa_final_mape_comp,tp_final_mape_comp]

names = ['clip_vitb16',  'dino_vitb',  'sam', 'nasa', 'twopoint']



Visualization(pr_results)
Visualization(pr_results_comps)

Visualization(best_pr_results, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/Clip_VitB16_PR.pdf",name = "2nd Order Polynomial Regression - Just Images", names = names)
Visualization(best_pr_results_comps, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/Clip_VitB16_PR_COMP.pdf",name = "2nd Order Polynomial Regression - Both", names = names)

In [ ]:

Visualize_Mean(pr_results)
Visualize_Mean(pr_results_comps)

In [ ]:

Visualize_Mean(pr_results)
Visualize_Mean(pr_results_comps)

best_pr_results = [clip_vitb16_final_mape,dino_vitb_final_mape, sam_final_mape,nasa_final_mape,tp_final_mape]
best_pr_results_comps = [clip_vitb16_final_mape_comp,dino_vitb_final_mape_comp, sam_final_mape_comp,nasa_final_mape_comp,tp_final_mape_comp]

names = ['clip_vitb16',  'dino_vitb',  'sam', 'nasa', 'twopoint']

Visualize_Mean(best_pr_results, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/PR_alt.pdf",Name = "2nd Order Polynomial Regression - Just Images", names = names)
Visualize_Mean(best_pr_results_comps, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/PR_COMP_alt.pdf",Name = "2nd Order Polynomial Regression - Both", names = names)

In [14]:
#Just Images
clip_vitb16_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_lr_32.npy')
clip_vitb32_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_lr_20.npy')
clip_vitl14_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_lr_42.npy')
dino_vits_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_lr_47.npy')
dino_vitb_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_lr_33.npy')
dino_vitl_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_lr_44.npy')
dino_vitg_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_lr_41.npy')
sam_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_lr_4.npy')
nasa_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_lr_2.npy')
tp_lr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_lr_1.npy')

#Images + Compositions
clip_vitb16_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_lr_comp_32.npy')
clip_vitb32_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_lr_comp_18.npy')
clip_vitl14_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_lr_comp_43.npy')
dino_vits_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_lr_comp_50.npy')
dino_vitb_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_lr_comp_33.npy')
dino_vitl_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_lr_comp_41.npy')
dino_vitg_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_lr_comp_36.npy')
sam_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_lr_comp_27.npy')
nasa_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_lr_comp_3.npy')
tp_lr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_lr_comp_5.npy')

In [ ]:
print ("Just Images")
print("CLIP VitB16",mean(clip_vitb16_lr))
print("CLIP VitB32",mean(clip_vitb32_lr))
print("CLIP VitL14",mean(clip_vitl14_lr))
print("DINO Vits",mean(dino_vits_lr))
print("DINO VitB",mean(dino_vitb_lr))
print("DINO VitL",mean(dino_vitl_lr))
print("DINO VitG",mean(dino_vitg_lr))
print("SAM",mean(sam_lr))
print("NASA",mean(nasa_lr))
print("Two Point",mean(tp_lr))

print ("Images & Compositions")
print("CLIP VitB16",mean(clip_vitb16_lr_comp))
print("CLIP VitB32",mean(clip_vitb32_lr_comp))
print("CLIP VitL14",mean(clip_vitl14_lr_comp))
print("DINO Vits",mean(dino_vits_lr_comp))
print("DINO VitB",mean(dino_vitb_lr_comp))
print("DINO VitL",mean(dino_vitl_lr_comp))
print("DINO VitG",mean(dino_vitg_lr_comp))
print("SAM",mean(sam_lr_comp))
print("NASA",mean(nasa_lr_comp))
print("Two Point",mean(tp_lr_comp))

In [ ]:
lr_results = [clip_vitb16_lr,clip_vitb32_lr,clip_vitl14_lr,dino_vits_lr,dino_vitb_lr, dino_vitl_lr, dino_vitg_lr, sam_lr,nasa_lr,tp_lr]
lr_results_comps = [clip_vitb16_lr_comp,clip_vitb32_lr_comp,clip_vitl14_lr_comp,dino_vits_lr_comp,dino_vitb_lr_comp, dino_vitl_lr_comp, dino_vitg_lr_comp, sam_lr_comp,nasa_lr_comp,tp_lr_comp]

best_lr_results = [clip_vitb16_lr,dino_vitl_lr, sam_lr,nasa_lr,tp_lr]
best_lr_results_comps = [clip_vitl14_lr_comp, dino_vitl_lr_comp, sam_lr_comp,nasa_lr_comp,tp_lr_comp]


names1 = ['clip_vitb16',  'dino_vitl',  'sam', 'nasa', 'twopoint']
names2 = ['clip_vitl14',  'dino_vitl',  'sam', 'nasa', 'twopoint']

Visualization(lr_results)
Visualization(lr_results_comps)

Visualization(best_lr_results, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/LR.pdf",name = "Linear Regression - Just Images", names = names1)
Visualization(best_lr_results_comps, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/LR_COMP.pdf",name = "Linear Regression - Both", names = names2)



In [ ]:
lr_results = [clip_vitb16_lr,clip_vitb32_lr,clip_vitl14_lr,dino_vits_lr,dino_vitb_lr, dino_vitl_lr, dino_vitg_lr, sam_lr,nasa_lr,tp_lr]
lr_results_comps = [clip_vitb16_lr_comp,clip_vitb32_lr_comp,clip_vitl14_lr_comp,dino_vits_lr_comp,dino_vitb_lr_comp, dino_vitl_lr_comp, dino_vitg_lr_comp, sam_lr_comp,nasa_lr_comp,tp_lr_comp]

best_lr_results = [clip_vitl14_lr,dino_vitl_lr, sam_lr,nasa_lr,tp_lr]
best_lr_results_comps = [clip_vitl14_lr_comp, dino_vitl_lr_comp, sam_lr_comp,nasa_lr_comp,tp_lr_comp]

names1 = ['clip_vitl14',  'dino_vitl',  'sam', 'nasa', 'twopoint']
names2 = ['clip_vitl14',  'dino_vitl',  'sam', 'nasa', 'twopoint']

Visualize_Mean(lr_results)
Visualize_Mean(lr_results_comps)

Visualize_Mean(best_lr_results, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/LR_alt.pdf",Name = "Linear Regression - Just Images", names = names1)
Visualize_Mean(best_lr_results_comps, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/LR_COMP_alt.pdf",Name = "Linear Regression - Both", names = names2)






In [15]:
#Just Images
clip_vitb16_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_svr_50.npy')
clip_vitb32_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_svr_50.npy')
clip_vitl14_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_svr_42.npy')
dino_vits_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_svr_14.npy')
dino_vitb_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_svr_30.npy')
dino_vitl_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_svr_34.npy')
dino_vitg_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_svr_37.npy')
sam_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_svr_1.npy')
nasa_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_svr_5.npy')
tp_svr = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_svr_5.npy')

#Images + Compositions
clip_vitb16_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb16_svr_comp_40.npy')
clip_vitb32_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitb32_svr_comp_23.npy')
clip_vitl14_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/clip_vitl14_svr_comp_46.npy')
dino_vits_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vits_svr_comp_42.npy')
dino_vitb_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitb_svr_comp_23.npy')
dino_vitl_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitl_svr_comp_49.npy')
dino_vitg_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/dino_vitg_svr_comp_38.npy')
sam_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/sam_svr_comp_3.npy')
nasa_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/nasa_svr_comp_4.npy')
tp_svr_comp = np.load('/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/tp_svr_comp_5.npy')

In [ ]:
print ("Just Images")
print("CLIP VitB16",mean(clip_vitb16_svr))
print("CLIP VitB32",mean(clip_vitb32_svr))
print("CLIP VitL14",mean(clip_vitl14_svr))
print("DINO Vits",mean(dino_vits_svr))
print("DINO VitB",mean(dino_vitb_svr))
print("DINO VitL",mean(dino_vitl_svr))
print("DINO VitG",mean(dino_vitg_svr))
print("SAM",mean(sam_svr))
print("NASA",mean(nasa_svr))
print("Two Point",mean(tp_svr))

print ("Images & Compositions")
print("CLIP VitB16",mean(clip_vitb16_svr_comp))
print("CLIP VitB32",mean(clip_vitb32_svr_comp))
print("CLIP VitL14",mean(clip_vitl14_svr_comp))
print("DINO Vits",mean(dino_vits_svr_comp))
print("DINO VitB",mean(dino_vitb_svr_comp))
print("DINO VitL",mean(dino_vitl_svr_comp))
print("DINO VitG",mean(dino_vitg_svr_comp))
print("SAM",mean(sam_svr_comp))
print("NASA",mean(nasa_svr_comp))
print("Two Point",mean(tp_svr_comp))

In [ ]:
svr_results = [clip_vitb16_svr,clip_vitb32_svr,clip_vitl14_svr,dino_vits_svr,dino_vitb_svr, dino_vitl_svr, dino_vitg_svr, sam_svr,nasa_svr,tp_svr]
svr_results_comps = [clip_vitb16_svr_comp,clip_vitb32_svr_comp,clip_vitl14_svr_comp,dino_vits_svr_comp,dino_vitb_svr_comp, dino_vitl_svr_comp, dino_vitg_svr_comp, sam_svr_comp,nasa_svr_comp,tp_svr_comp]

best_svr_results = [clip_vitl14_svr,dino_vitg_svr, sam_svr,nasa_svr,tp_svr]
best_svr_results_comps = [clip_vitb16_svr_comp, dino_vitg_svr_comp, sam_svr_comp,nasa_svr_comp,tp_svr_comp]


names1 = ['clip_vitl14',  'dino_vitg',  'sam', 'nasa', 'twopoint']
names2 = ['clip_vitb16',  'dino_vitg',  'sam', 'nasa', 'twopoint']

Visualization(svr_results)
Visualization(svr_results_comps)

Visualization(best_svr_results, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/SVR.pdf",name = "Support Vector Regression - Just Images", names = names1)
Visualization(best_svr_results_comps, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/SVR_COMP.pdf",name = "Support Vector Regression - Both", names = names2)


In [ ]:
Visualization(svr_results)
Visualization(svr_results_comps)

In [ ]:
svr_results = [clip_vitb16_svr,clip_vitb32_svr,clip_vitl14_svr,dino_vits_svr,dino_vitb_svr, dino_vitl_svr, dino_vitg_svr, sam_svr,nasa_svr,tp_svr]
svr_results_comps = [clip_vitb16_svr_comp,clip_vitb32_svr_comp,clip_vitl14_svr_comp,dino_vits_svr_comp,dino_vitb_svr_comp, dino_vitl_svr_comp, dino_vitg_svr_comp, sam_svr_comp,nasa_svr_comp,tp_svr_comp]

best_svr_results = [clip_vitb16_svr,dino_vitl_svr, sam_svr,nasa_svr,tp_svr]
best_svr_results_comps = [clip_vitb16_svr_comp, dino_vitl_svr_comp, sam_svr_comp,nasa_svr_comp,tp_svr_comp]


names1 = ['clip_vitb16',  'dino_vitl',  'sam', 'nasa', 'twopoint']
names2 = ['clip_vitb16',  'dino_vitl',  'sam', 'nasa', 'twopoint']

Visualize_Mean(svr_results)
Visualize_Mean(svr_results_comps)

Visualize_Mean(best_svr_results, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/SVR_alt.pdf",Name = "Support Vector Regression - Just Images", names = names1)
Visualize_Mean(best_svr_results_comps, save = True, filename = "/content/drive/MyDrive/11_12/Experimental Dataset/12_6_Results/SVR_COMP_alt.pdf",Name = "Support Vector Regression - Both", names = names2)


